# Part 2: Basic LLM Chat Tool

## Introduction

In this part, you'll create a simple command-line chat tool that interacts with a Large Language Model (LLM) through the Hugging Face API. This tool will allow you to have conversations with an LLM about healthcare topics.

## Learning Objectives

- Connect to the Hugging Face API
- Create a basic interactive chat loop
- Handle simple error cases
- Test with healthcare questions

## Setup and Installation


In [2]:
# Install required packages
%pip install -r requirements.txt

# Additional packages for LLM API interaction
%pip install requests

# Import necessary libraries
import os
import sys
import requests
import time
import logging
import argparse
from typing import Optional

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Create directories
os.makedirs('utils', exist_ok=True)
os.makedirs('results/part_2', exist_ok=True)

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## 1. Connecting to the Hugging Face API

The Hugging Face Inference API provides access to many language models. We'll use models that are available on the free tier.

In [ ]:
# Example of a simple API request to Hugging Face
API_URL = "https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta"
#https://router.huggingface.co/hf-inference/models/HuggingFaceH4/zephyr-7b-beta/v1/chat/completions
headers = {"Authorization": "Bearer "} # Optional for some models


logging.basicConfig(level=logging.INFO)

def query(payload):
    """
    Send a query to the Hugging Face API
    
    Args:
        payload: Dictionary containing the query parameters
        
    Returns:
        The API response
    """
    # TODO: Implement the API request
    # Use requests.post to send the query to the API_URL
    # Return the response

    try:
        response = requests.post(API_URL, headers=headers, json=payload, timeout=50)
        response.raise_for_status()
        output = response.json()
        if isinstance(output, list) and 'generated_text' in output[0]:
            return output[0]['generated_text']
        elif isinstance(output, dict) and 'generated_text' in output:
            return output['generated_text']
        return str(output)
    except requests.exceptions.RequestException as e:
        logging.error(f"Error during API request: {e}")
        return "Error: Unable to connect to model."
    #pass
    
# Test the query function
test_payload = {"inputs": "What are the symptoms of diabetes?"}
response = query(test_payload)
print(response)

2025-05-21 12:27:23,946 - root - ERROR - Error during API request: 402 Client Error: Payment Required for url: https://router.huggingface.co/hf-inference/models/HuggingFaceH4/zephyr-7b-beta/v1/chat/completions


Error: Unable to connect to model.


## 2. Creating Simple Chat Scripts

Your task is to create two simple scripts that interact with the Hugging Face API:

1. A basic one-off chat script (`utils/one_off_chat.py`)
2. A contextual conversation script (`utils/conversation.py`)

### One-Off Chat Script

Create a script that handles independent interactions (each prompt/response is separate):

In [ ]:
# utils/one_off_chat.py

import requests
import argparse
import os

def get_response(prompt, model_name="HuggingFaceH4/zephyr-7b-beta", api_key=None):
    """
    Get a response from the model
    
    Args:
        prompt: The prompt to send to the model
        model_name: Name of the model to use
        api_key: API key for authentication (optional for some models)
        
    Returns:
        The model's response
    """
    # TODO: Implement the get_response function
    # Set up the API URL and headers
    api_url = f"https://api-inference.huggingface.co/models/{model_name}"
    headers = {"Authorization": f"Bearer {api_key}"} if api_key else {}
    # Create a payload with the prompt
    payload = {"inputs": prompt}
    try:
        # Send the payload to the API
        response = requests.post(api_url, headers=headers, json=payload, timeout=50)
        response.raise_for_status()

        # Extract and return the generated text from the response
        output = response.json()
        if isinstance(output, list) and 'generated_text' in output[0]:
            return output[0]['generated_text']
        elif isinstance(output, dict) and 'generated_text' in output:
            return output['generated_text']
        return str(output)
 # Handle any errors that might occur
    except requests.exceptions.RequestException as e:
        return f"[ERROR] API request failed: {e}"


    #pass

def run_chat(model_name, api_key):
    """Run an interactive chat session"""
    print("Welcome to the Simple LLM Chat! Type 'exit' to quit.")
    
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == 'exit':
            print("Goodbye!")
            break
            
        # TODO: Get response from the model
        response = get_response(user_input, model_name=model_name, api_key=api_key)

        # Print the response
        print(f"LLM: {response}")
        
def main():
    parser = argparse.ArgumentParser(description="Chat with an LLM")
    # TODO: Add arguments to the parser
    parser.add_argument("--model", type=str, default="HuggingFaceH4/zephyr-7b-beta", help="Hugging Face model name")
    parser.add_argument("--api_key", type=str, default=os.getenv("HF_API_KEY"), help="Hugging Face API token")

    args = parser.parse_args()
    
    # TODO: Run the chat function with parsed arguments
    run_chat(model_name=args.model, api_key=args.api_key)
    
if __name__ == "__main__":
    main()


usage: ipykernel_launcher.py [-h] [--api_key API_KEY]
ipykernel_launcher.py: error: unrecognized arguments: --f=/Users/jenniferzhu/Library/Jupyter/runtime/kernel-v3b8060c47f506d76fbf1e59ba1e7a351775d4ff08.json


SystemExit: 2

/Users/jenniferzhu/Library/Python/3.12/lib/python/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# utils/one_off_chat.py no API 

from transformers import pipeline
import argparse

def get_response(prompt, model_name):
    """
    Run the model locally using transformers.pipeline

    Args:
        prompt: The input text to send to the model
        model_name: Name of the Hugging Face model to load

    Returns:
        The generated text from the model, or error message
    """
    try:
        # Force use of PyTorch framework to avoid TF model loading errors
        chat = pipeline("text2text-generation", model=model_name, framework="pt")
        response = chat(prompt, max_length=100, do_sample=False)
        return response[0]["generated_text"]
    except Exception as e:
        return f"Error: {str(e)}"

def run_chat(model_name):
    """
    Run an interactive chat session
    """
    print("Local LLM Chat (no API). Type 'exit' to quit.")

    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == 'exit':
            print("Goodbye!")
            break

        response = get_response(user_input, model_name=model_name)
        print(f"Bot: {response}")

def main():
    parser = argparse.ArgumentParser(description="Chat with a local LLM")
    
    # Add model name argument
    parser.add_argument(
        "--model_name",
        type=str,
        default="google/flan-t5-small",
        help="Name of the Hugging Face model to use locally"
    )

    args = parser.parse_args()
    run_chat(model_name=args.model_name)

if __name__ == "__main__":
    main()


### Contextual Conversation Script

Create a script that maintains conversation history:


In [ ]:
# utils/conversation.py

import requests
import argparse
import os

def get_response(prompt, history=None, model_name="google/flan-t5-base", api_key=None, history_length=3):
    """
    Get a response from the model using conversation history
    
    Args:
        prompt: The current user prompt
        history: List of previous (prompt, response) tuples
        model_name: Name of the model to use
        api_key: API key for authentication
        history_length: Number of previous exchanges to include in context
        
    Returns:
        The model's response
    """
    # TODO: Implement the contextual response function
    # Initialize history if None
    if history is None:
        history = []
        
    # TODO: Format a prompt that includes previous exchanges
    context_prompt = ""
    for past_prompt, past_response in history[-history_length:]:
        context_prompt += f"User: {past_prompt}\nAssistant: {past_response}\n"
    context_prompt += f"User: {prompt}\nAssistant:"
    # Get a response from the API
    # Return the response
    api_url = f"https://api-inference.huggingface.co/models/{model_name}"
    headers = {"Authorization": f"Bearer {api_key}"} if api_key else {}
    payload = {"inputs": context_prompt}

    try:
        response = requests.post(api_url, headers=headers, json=payload, timeout=30)
        response.raise_for_status()
        output = response.json()
        if isinstance(output, list) and 'generated_text' in output[0]:
            return output[0]['generated_text'].strip()
        elif isinstance(output, dict) and 'generated_text' in output:
            return output['generated_text'].strip()
        return str(output)
    except requests.exceptions.RequestException as e:
        return f"[ERROR] API request failed: {e}"
    #pass

def run_chat(model_name, api_key, history_length):
    """Run an interactive chat session with context"""
    print("Welcome to the Contextual LLM Chat! Type 'exit' to quit.")
    
    # Initialize conversation history
    history = []
    
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == 'exit':
            print("Goodbye!")
            break
            
        # TODO: Get response using conversation history
        response = get_response(
            prompt=user_input,
            history=history,
            model_name=model_name,
            api_key=api_key,
            history_length=history_length
        )
        # Update history
        # Print the response
        print(f"LLM: {response}\n")
        history.append((user_input, response))
        
def main():
    parser = argparse.ArgumentParser(description="Chat with an LLM using conversation history")
    # TODO: Add arguments to the parser
    parser = argparse.ArgumentParser(description="Chat with an LLM using conversation history")
    parser.add_argument("--model", type=str, default="HuggingFaceH4/zephyr-7b-beta", help="Hugging Face model name")
    parser.add_argument("--api_key", type=str, default=os.getenv("HF_API_KEY"), help="Hugging Face API token")
    parser.add_argument("--history_length", type=int, default=3, help="Number of previous turns to include in context")

    
    args = parser.parse_args()
    
    # TODO: Run the chat function with parsed arguments
    run_chat(model_name=args.model, api_key=args.api_key, history_length=args.history_length)

    
if __name__ == "__main__":
    main()

## 3. Testing and Evaluation

Create a script to test your chat implementations with specific healthcare questions.

In [ ]:
# utils/test_chat.py

import os
import csv
from pathlib import Path

# Import our chat modules - since we're in the same directory
from one_off_chat import get_response as get_one_off_response
# Optionally import the conversation module if testing that too
# from conversation import get_response as get_contextual_response

def test_chat(questions, model_name="google/flan-t5-base", api_key=None):
    """
    Test the chat function with a list of questions
    
    Args:
        questions: A list of questions to test
        model_name: Name of the model to use
        api_key: API key for authentication
        
    Returns:
        A dictionary mapping questions to responses
    """
    results = {}
    
    for question in questions:
        print(f"Testing question: {question}")
        # Get response using the one-off chat function
        response = get_one_off_response(question, model_name, api_key)
        results[question] = response
        
    return results

# List of healthcare questions to test
test_questions = [
    "What are the symptoms of gout?",
    "How is gout diagnosed?",
    "What treatments are available for gout?",
    "What lifestyle changes can help manage gout?",
    "What foods should be avoided with gout?"
]

def save_results(results, output_file="results/part_2/example.txt"):
    """
    Save the test results to a file
    
    Args:
        results: Dictionary mapping questions to responses
        output_file: Path to the output file
    """
    with open(output_file, 'w') as f:
        # Write header
        f.write("# LLM Chat Tool Test Results\n\n")
        
        # Write usage examples
        f.write("## Usage Examples\n\n")
        f.write("```bash\n")
        f.write("# Run the one-off chat\n")
        f.write("python utils/one_off_chat.py\n\n")
        f.write("# Run the contextual chat\n")
        f.write("python utils/conversation.py\n")
        f.write("```\n\n")
        
        # Write test results
        f.write("## Test Results\n\n")
        f.write("```csv\n")
        f.write("question,response\n")
        
        for question, response in results.items():
            # Format the question and response for CSV
            q = question.replace(',', '').replace('\n', ' ')
            r = response.replace(',', '').replace('\n', ' ')
            f.write(f"{q},{r}\n")
            
        f.write("```\n")

# Run the test and save results
if __name__ == "__main__":
    results = test_chat(test_questions)
    save_results(results)
    print("Test results saved to results/part_2/example.txt")